In [1]:
import sys
import os
lib = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities\droplet_dataset'
lib2 = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities'
lib3 = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\data_analysis'
lib4 = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy'
lib5 = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\scripts'
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)



In [2]:
import os
from os.path import join
import sklearn
from droplet_dataset import *
from utilities import *
from matplotlib import pyplot
import numpy as np
import scipy
import pickle
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import random
from scipy.stats import pearsonr
from matplotlib.pyplot import figure
sklearn.__version__
%matplotlib notebook

In [3]:
def extract_sample(sample_id):
    sample_path = join(SAMPLES_PATH, sample_id, f'{sample_id}.pkl')
    rna_sample = pickle.load(open(sample_path, 'rb'))
    return rna_sample

def extract_sample_no_wrapper_folder(sample_id):
    sample_path = join(SAMPLES_PATH, sample_id)
    rna_sample = pickle.load(open(sample_path, 'rb'))
    return rna_sample


    
def get_informative_df(rna_sample):
    interesting_properties = ['is_apoptosis', 
                              'is_immune',
                              'is_cancer',
                              'is_stromal',
                              'is_epithelial',
                              'cancer_immune_conflict', 
                              'is_doublet', 
                              'is_lymphoid',
                              'is_myeloid',
                              'should_be_removed',
                             'is_CelBender_empty']
    relevant_cells = {key:[cf for cf in rna_sample.cells_information if getattr(cf, key)] for key in interesting_properties}
    n_cells = len(relevant_cells)
    counter = {key1:{key2: 0 for key2 in interesting_properties} for key1 in interesting_properties}
    for prop1 in interesting_properties:
        for prop2 in interesting_properties:
            for cell_inf in relevant_cells[prop1]:
                counter[prop1][prop2] += int(getattr(cell_inf, prop2))
    df = pd.DataFrame(counter.values(), index=counter.keys())[counter.keys()]
    return  df

def check_cross_cell_types(sample_id):
    sample_path = join(SAMPLES_PATH, sample_id, f'{sample_id}.pkl')
    types = ['None', 'Immune_general', 'T cells', 'CD4 helper T cells', 'CD8 Cytotoxic T cells', 'Regulatory T cells',
             'Regulatory CD4 T cells', 'Regulatory CD8 T cells', 'Regulatory CD4_CD8 T cells', 'NKT cells', 'NK cells',
             'B cells', 'Activated T cells', 'Senescence T cells', 'Terminal effector', 'Exhausted T cells',
             'Stem_like T cells', 'Memory T cells', 'Memory CD4 T cells', 'Memory CD8 T cells',
             'Memory CD4_CD8 T cells', 'Macrophage_immature', 'Macrophage_mature', 'Monocyte_immature',
             'Monocyte_mature', 'cDCs_dendritic_cells', 'pDCs', 'myeloid cells_general_immature',
             'myeloid cells_general_mature', 'Neutrophils', 'Granolocytes']
    table = {k1: {k2: 0 for k2 in types} for k1 in types}
    data = pickle.load(open(sample_path, 'rb'))
    cls = [list(set(v)) for v in data.cells_information.getattr('cell_type_list')]
    cls = [v if len(v) else ['None'] for v in cls]

    for cell in cls:
        for t1 in cell:
            for t2 in cell:
                table[t1][t2] += 1
    df = pd.DataFrame(table.values(), index=table.keys())[table.keys()]
    return df

def show_informative_table(ROW_SAMPLES_PATH, SAMPLES_INFORMATION_PATH, should_be_removed):
    not_investigate = []#['M133', 'M108']
    samples = [subfolder for subfolder in os.listdir(ROW_SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
    cross_check_df = None
    for sample_id in samples:
    #     print(sample)
        rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}'),
                                        cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}'))

        # REMOVING CELLS THAT SHOULD BE REMOVED
        if should_be_removed:
            rna_sample = rna_sample.filter_cells_by_property('should_be_removed', False)

        df = get_informative_df(rna_sample)
        if cross_check_df is None:
            cross_check_df = df
        else:
            cross_check_df = cross_check_df.add(df)

    return cross_check_df

# Investigate properties of selected sample
Specify sample name and samples path

In [8]:
sample_id = 'M120'
ROW_SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\4.3.21'


rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}.pkl'),
                                    cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}.pkl'))
rna_sample = rna_sample.filter_cells_by_property('should_be_removed', False)

In [9]:
get_informative_df(rna_sample)

,is_apoptosis,is_immune,is_cancer,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed
is_apoptosis,0,0,0,0,0,0,0,0
is_immune,0,1205,0,0,0,238,394,0
is_cancer,0,0,450,0,0,0,0,0
cancer_immune_conflict,0,0,0,0,0,0,0,0
is_doublet,0,0,0,0,0,0,0,0
is_lymphoid,0,238,0,0,0,238,163,0
is_myeloid,0,394,0,0,0,163,394,0
should_be_removed,0,0,0,0,0,0,0,0


# Investigate properties over all samples
Specify samples path and samples you're not interested in investigating them.

In [7]:
ROW_SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\4.3.21'
not_investigate = []#['M133', 'M108']


In [9]:
samples = [subfolder for subfolder in os.listdir(SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
cross_check_df = None
for sample_id in samples:
#     print(sample)
    rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}'),
                                    cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}'))
    
    # REMOVING CELLS THAT SHOULD BE REMOVED
    rna_sample = rna_sample.filter_cells_by_property('should_be_removed', False)
    
    df = get_informative_df(rna_sample)
    if cross_check_df is None:
        cross_check_df = df
    else:
        cross_check_df = cross_check_df.add(df)

cross_check_df

,is_apoptosis,is_immune,is_cancer,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed
is_apoptosis,0,0,0,0,0,0,0,0
is_immune,0,89024,0,0,81,60145,18480,0
is_cancer,0,0,109283,12058,5256,0,0,0
cancer_immune_conflict,0,0,12058,12058,1154,0,0,0
is_doublet,0,81,5256,1154,5374,47,52,0
is_lymphoid,0,60145,0,0,47,60145,796,0
is_myeloid,0,18480,0,0,52,796,18480,0
should_be_removed,0,0,0,0,0,0,0,0


In [12]:
# If you want to save result
OUTPATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\investigate_cells_propeties\samples_properties_16.12.20_empty_removed.xlsx'
cross_check_df.to_excel(OUTPATH)

# Cross-checking cell-types of selected sample

In [48]:
SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\scrublet\5.12.20'
sample_id = 'M108'

In [52]:
check_cross_cell_types(sample_id).head()

,None,Immune_general,T cells,CD4 helper T cells,CD8 Cytotoxic T cells,Regulatory T cells,Regulatory CD4 T cells,Regulatory CD8 T cells,Regulatory CD4_CD8 T cells,NKT cells,...,Macrophage_immature,Macrophage_mature,Monocyte_immature,Monocyte_mature,cDCs_dendritic_cells,pDCs,myeloid cells_general_immature,myeloid cells_general_mature,Neutrophils,Granolocytes
None,7314,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Immune_general,0,2946,1398,135,705,30,6,0,0,57,...,3,24,3,87,120,0,0,135,651,159
T cells,0,1398,1398,135,705,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CD4 helper T cells,0,135,135,135,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CD8 Cytotoxic T cells,0,705,705,15,705,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Cross-checking cell-types over all samples

In [9]:
SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\scrublet\16.12.20_empty_removed'
not_investigate = [] #['M133', 'M108']

In [14]:
samples = [subfolder for subfolder in os.listdir(SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
cell_types_df = None
for sample in samples:
#     print(sample)
    df = check_cross_cell_types(sample)
    if cell_types_df is None:
        cell_types_df = df
    else:
        cell_types_df = cell_types_df.add(df)

cell_types_df

,None,Immune_general,T cells,CD4 helper T cells,CD8 Cytotoxic T cells,Regulatory T cells,Regulatory CD4 T cells,Regulatory CD8 T cells,Regulatory CD4_CD8 T cells,NKT cells,...,Macrophage_immature,Macrophage_mature,Monocyte_immature,Monocyte_mature,cDCs_dendritic_cells,pDCs,myeloid cells_general_immature,myeloid cells_general_mature,Neutrophils,Granolocytes
None,164285,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Immune_general,0,50565,24250,2358,12195,454,257,37,28,1112,...,120,1936,216,3739,2327,54,44,3249,9917,583
T cells,0,24250,24250,2358,12195,2,1,0,0,64,...,0,0,4,0,0,0,0,0,0,0
CD4 helper T cells,0,2358,2358,2358,134,0,1,0,0,5,...,0,0,2,0,0,0,0,0,0,0
CD8 Cytotoxic T cells,0,12195,12195,134,12195,0,0,0,0,32,...,0,0,1,0,0,0,0,0,0,0
Regulatory T cells,0,454,2,0,0,454,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Regulatory CD4 T cells,0,257,1,1,0,0,257,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Regulatory CD8 T cells,0,37,0,0,0,0,0,37,0,0,...,0,0,0,0,0,0,0,0,0,0
Regulatory CD4_CD8 T cells,0,28,0,0,0,0,0,0,28,0,...,0,0,0,0,0,0,0,0,0,0
NKT cells,0,1112,64,5,32,1,0,0,0,1112,...,0,0,0,0,0,0,0,0,0,0


In [1]:
# If you want to save result
# OUTPATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\investigate_cells_propeties\samples_cell_types_16.12.20_empty_removed.xlsx'
# cell_types_df.to_excel(OUTPATH)

# ALL Cohort side-by-side

### 4.3.21

In [22]:
# ROW_SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
# SAMPLES_INFORMATION_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\4.3.21'
# should_be_removed = True
# show_informative_table(ROW_SAMPLES_PATH, SAMPLES_INFORMATION_PATH, should_be_removed)

### 10.5.21
1. considering cells for removal
2. exclusing cells for removal

In [23]:
# ROW_SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
# SAMPLES_INFORMATION_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\10.5.21'
# should_be_removed = False
# show_informative_table(ROW_SAMPLES_PATH, SAMPLES_INFORMATION_PATH, should_be_removed)

In [24]:
# ROW_SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
# SAMPLES_INFORMATION_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\10.5.21'
# should_be_removed = True
# show_informative_table(ROW_SAMPLES_PATH, SAMPLES_INFORMATION_PATH, should_be_removed)

### 10.6.21
1. considering cells for removal
2. exclusing cells for removal

In [19]:
ROW_SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\10.6.21'
should_be_removed = False
show_informative_table(ROW_SAMPLES_PATH, SAMPLES_INFORMATION_PATH, should_be_removed)

,is_apoptosis,is_immune,is_cancer,is_stromal,is_epithelial,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed,is_CelBender_empty
is_apoptosis,49517,1708,27785,13622,0,860,4978,639,350,49517,11431
is_immune,1708,90492,0,0,0,0,1443,62231,16903,6831,35513
is_cancer,27785,0,132904,0,0,0,8696,0,0,27969,19834
is_stromal,13622,0,0,45381,0,0,1318,0,0,13824,13485
is_epithelial,0,0,0,0,1187,0,2,0,0,0,769
cancer_immune_conflict,860,0,0,0,0,23854,1384,0,0,23854,4313
is_doublet,4978,1443,8696,1318,2,1384,12874,1110,253,7915,460
is_lymphoid,639,62231,0,0,0,0,1110,62231,817,3079,24974
is_myeloid,350,16903,0,0,0,0,253,817,16903,3228,5053
should_be_removed,49517,6831,27969,13824,0,23854,7915,3079,3228,78020,16595


In [18]:
ROW_SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\10.6.21'
should_be_removed = True
show_informative_table(ROW_SAMPLES_PATH, SAMPLES_INFORMATION_PATH, should_be_removed)

,is_apoptosis,is_immune,is_cancer,is_stromal,is_epithelial,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed,is_CelBender_empty
is_apoptosis,0,0,0,0,0,0,0,0,0,0,0
is_immune,0,83661,0,0,0,0,45,59152,13675,0,33588
is_cancer,0,0,104935,0,0,0,4874,0,0,0,15066
is_stromal,0,0,0,31557,0,0,38,0,0,0,9661
is_epithelial,0,0,0,0,1187,0,2,0,0,0,769
cancer_immune_conflict,0,0,0,0,0,0,0,0,0,0,0
is_doublet,0,45,4874,38,2,0,4959,14,17,0,9
is_lymphoid,0,59152,0,0,0,0,14,59152,0,0,23983
is_myeloid,0,13675,0,0,0,0,17,0,13675,0,4364
should_be_removed,0,0,0,0,0,0,0,0,0,0,0


### 26.6.21

1. considering cells for removal
2. excluding cells for removal

In [16]:
ROW_SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\26.6.21'
should_be_removed = False
show_informative_table(ROW_SAMPLES_PATH, SAMPLES_INFORMATION_PATH, should_be_removed)

,is_apoptosis,is_immune,is_cancer,is_stromal,is_epithelial,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed,is_CelBender_empty
is_apoptosis,49517,1708,27785,13622,0,860,4978,639,350,49517,11431
is_immune,1708,86331,0,0,0,0,1443,61827,15253,6831,35038
is_cancer,27785,0,132904,0,0,0,8696,0,0,27969,19834
is_stromal,13622,0,0,50729,5348,0,1320,0,0,13824,14729
is_epithelial,0,0,0,5348,5348,0,2,0,0,0,1244
cancer_immune_conflict,860,0,0,0,0,23854,1384,0,0,23854,4313
is_doublet,4978,1443,8696,1320,2,1384,12874,1110,253,7915,460
is_lymphoid,639,61827,0,0,0,0,1110,61827,817,3079,24950
is_myeloid,350,15253,0,0,0,0,253,817,15253,3228,4964
should_be_removed,49517,6831,27969,13824,0,23854,7915,3079,3228,78020,16595


In [17]:
ROW_SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\26.6.21'
should_be_removed = True
show_informative_table(ROW_SAMPLES_PATH, SAMPLES_INFORMATION_PATH, should_be_removed)

,is_apoptosis,is_immune,is_cancer,is_stromal,is_epithelial,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed,is_CelBender_empty
is_apoptosis,0,0,0,0,0,0,0,0,0,0,0
is_immune,0,79500,0,0,0,0,45,58748,12025,0,33113
is_cancer,0,0,104935,0,0,0,4874,0,0,0,15066
is_stromal,0,0,0,36905,5348,0,40,0,0,0,10905
is_epithelial,0,0,0,5348,5348,0,2,0,0,0,1244
cancer_immune_conflict,0,0,0,0,0,0,0,0,0,0,0
is_doublet,0,45,4874,40,2,0,4959,14,17,0,9
is_lymphoid,0,58748,0,0,0,0,14,58748,0,0,23959
is_myeloid,0,12025,0,0,0,0,17,0,12025,0,4275
should_be_removed,0,0,0,0,0,0,0,0,0,0,0


### 4.11.21 (M97-M146)

1. considering cells for removal
2. excluding cells for removal

In [13]:
ROW_SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\4.11.21'
should_be_removed = False
show_informative_table(ROW_SAMPLES_PATH, SAMPLES_INFORMATION_PATH, should_be_removed)

,is_apoptosis,is_immune,is_cancer,is_stromal,is_epithelial,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed,is_CelBender_empty
is_apoptosis,49517,1708,27785,13622,0,860,4978,639,350,49517,11431
is_immune,1708,86331,0,0,0,0,1443,61827,15253,12903,35038
is_cancer,27785,0,132904,0,0,0,8696,0,0,41617,19834
is_stromal,13622,0,0,50729,5348,0,1320,0,0,22666,14729
is_epithelial,0,0,0,5348,5348,0,2,0,0,418,1244
cancer_immune_conflict,860,0,0,0,0,23854,1384,0,0,23854,4313
is_doublet,4978,1443,8696,1320,2,1384,12874,1110,253,8148,460
is_lymphoid,639,61827,0,0,0,0,1110,61827,817,6829,24950
is_myeloid,350,15253,0,0,0,0,253,817,15253,3336,4964
should_be_removed,49517,12903,41617,22666,418,23854,8148,6829,3336,106582,29364


In [12]:
ROW_SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\4.11.21'
should_be_removed = True
show_informative_table(ROW_SAMPLES_PATH, SAMPLES_INFORMATION_PATH, should_be_removed)

,is_apoptosis,is_immune,is_cancer,is_stromal,is_epithelial,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed,is_CelBender_empty
is_apoptosis,0,0,0,0,0,0,0,0,0,0,0
is_immune,0,73428,0,0,0,0,42,54998,11917,0,30534
is_cancer,0,0,91287,0,0,0,4676,0,0,0,9573
is_stromal,0,0,0,28063,4930,0,8,0,0,0,6208
is_epithelial,0,0,0,4930,4930,0,1,0,0,0,909
cancer_immune_conflict,0,0,0,0,0,0,0,0,0,0,0
is_doublet,0,42,4676,8,1,0,4726,12,17,0,0
is_lymphoid,0,54998,0,0,0,0,12,54998,0,0,22570
is_myeloid,0,11917,0,0,0,0,17,0,11917,0,4202
should_be_removed,0,0,0,0,0,0,0,0,0,0,0


### New data_3.10.21 (only, M147-M173)) 4.11.21

1. considering cells for removal
2. excluding cells for removal

In [14]:
ROW_SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\Data\droplet_seq\new_data_3.10.21\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\outputs\new_data_3.10.21_outputs\inferCNV\update_runs\4.11.21'
should_be_removed = False
show_informative_table(ROW_SAMPLES_PATH, SAMPLES_INFORMATION_PATH, should_be_removed)

,is_apoptosis,is_immune,is_cancer,is_stromal,is_epithelial,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed,is_CelBender_empty
is_apoptosis,14161,933,7823,2607,0,1406,187,353,318,14161,0
is_immune,933,89837,0,0,0,0,3354,62187,14940,18688,0
is_cancer,7823,0,77479,0,0,0,540,0,0,11033,0
is_stromal,2607,0,0,13204,0,0,196,0,0,7370,0
is_epithelial,0,0,0,0,0,0,0,0,0,0,0
cancer_immune_conflict,1406,0,0,0,0,18671,1592,0,0,18671,0
is_doublet,187,3354,540,196,0,1592,5707,1177,1915,3738,0
is_lymphoid,353,62187,0,0,0,0,1177,62187,531,13041,0
is_myeloid,318,14940,0,0,0,0,1915,531,14940,1646,0
should_be_removed,14161,18688,11033,7370,0,18671,3738,13041,1646,57243,0


In [15]:
ROW_SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\Data\droplet_seq\new_data_3.10.21\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\outputs\new_data_3.10.21_outputs\inferCNV\update_runs\4.11.21'
should_be_removed = True
show_informative_table(ROW_SAMPLES_PATH, SAMPLES_INFORMATION_PATH, should_be_removed)

,is_apoptosis,is_immune,is_cancer,is_stromal,is_epithelial,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed,is_CelBender_empty
is_apoptosis,0,0,0,0,0,0,0,0,0,0,0
is_immune,0,71149,0,0,0,0,1950,49146,13294,0,0
is_cancer,0,0,66446,0,0,0,2,0,0,0,0
is_stromal,0,0,0,5834,0,0,17,0,0,0,0
is_epithelial,0,0,0,0,0,0,0,0,0,0,0
cancer_immune_conflict,0,0,0,0,0,0,0,0,0,0,0
is_doublet,0,1950,2,17,0,0,1969,83,1729,0,0
is_lymphoid,0,49146,0,0,0,0,83,49146,0,0,0
is_myeloid,0,13294,0,0,0,0,1729,0,13294,0,0
should_be_removed,0,0,0,0,0,0,0,0,0,0,0


# Debug

In [54]:
df.n_cells.iloc[:46].sum()
df.n_cells.sum()
# df.n_empty.sum()

500032

In [45]:
# rna_sample.counts.shape




183

In [49]:
# ROW_SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\Data\droplet_seq\ROW_DATA'
# SAMPLES_INFORMATION_PATH = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\outputs\inferCNV\update_runs\26.6.21'

ROW_SAMPLES_PATH = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\Data\droplet_seq\new_data_3.10.21\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\outputs\new_data_3.10.21_outputs\inferCNV\update_runs\4.11.21'



# sample_id = 'M100.pkl'
# rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}'),
#                                         cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}'))



In [50]:
samples = [subfolder for subfolder in os.listdir(ROW_SAMPLES_PATH) if subfolder.startswith('M')]
# df = pd.DataFrame(columns=['id', 'n_cells', 'n_empty'])

for sample_id in samples:
    print(sample_id)
    rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}'),
                                        cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}'))

    n_empty = sum(np.sum(rna_sample.counts > 0, axis=1) < 500)
    df = df.append(pd.DataFrame([[sample_id[:-4], rna_sample.number_of_cells, n_empty]], columns=df.columns))
df

M147.pkl
M148.pkl
M149.pkl
M150.pkl
M151.pkl
M152.pkl
M153.pkl
M154.pkl
M155.pkl
M156.pkl
M157.pkl
M158.pkl
M159.pkl
M160.pkl
M161.pkl
M162.pkl
M163.pkl
M164.pkl
M167.pkl
M168.pkl
M169.pkl
M170.pkl
M171.pkl
M172.pkl
M173.pkl


,id,n_cells,n_empty
0,M100,4055,651
0,M101,5141,523
0,M102,22694,5015
0,M103,2453,620
0,M104,4944,1355
...,...,...,...
0,M169,20446,6557
0,M170,6396,662
0,M171,7804,906
0,M172,10681,6247


In [3]:
def show_informative_table(ROW_SAMPLES_PATH, SAMPLES_INFORMATION_PATH, should_be_removed):
    not_investigate = []#['M133', 'M108']
    samples = [subfolder for subfolder in os.listdir(ROW_SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
    cross_check_df = None
    for sample_id in samples:
    #     print(sample)
        rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}'),
                                        cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}'))


In [21]:
path = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\classifying_cell_types\5.12.20'
sample_id = r'M132'
sample_path = join(path, sample_id, f'{sample_id}.pkl')
data = pickle.load(open(sample_path, 'rb'))

In [ ]:
(817 + 23854)

In [15]:
(817 + 23854 + 531 + 18671)/(299360+200672)

0.08774038461538461

In [16]:
(299360+200672)

500032

In [17]:
(49517 + 14161)/(299360+200672)

0.1273478497376168

In [7]:
samples = [subfolder for subfolder in os.listdir(SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
all_df = None
count = 0
for sample in samples:
    print(sample)
    sample_path = join(SAMPLES_PATH, sample, f'{sample}.pkl')
    data = pickle.load(open(sample_path, 'rb'))
    xx = sum([c.is_immune for c in data.cells_information])
    print(xx)
    count += xx
    
    yy = [1 for c in data.cells_information if c.is_immune == False and  len(c.cell_type_list)>0]
    print(sum(yy))
print(count)

M100
190
0
M101
1467
0
M102
734
0
M103
433
0
M104
277
0
M105
3094
0
M106
562
0
M107
1627
0
M108
116
0
M109
2121
0
M110
1869
0
M111
5658
0
M112
178
0
M114
4530
0
M115
2910
0
M116
3380
0
M118
1023
0
M120
1219
0
M121
4071
0
M122
676
0
M123
3368
0
M124
4274
0
M125
140
0
M126
2757
0
M127
78
0
M128
422
0
M129
527
0
M130
3273
0
M131
3986
0
M132
2663
0
M133
83
0
M134
1798
0
M135
1955
0
M136
1268
0
M137
3758
0
M138
2428
0
M139
2428
0
M140
3395
0
M141
85
0
M143
1168
0
M144
538
0
M145
3983
0
M146
5783
0
M97
1299
0
M98
760
0
M99
3296
0
91648


In [15]:
import pandas as pd
import numpy as np

In [29]:
df = pd.DataFrame(np.array([[1,2,3,4], ['One', 'Two', 'Three', 'Four']]).T, columns=['Num', 'word'])
df

,Num,word
0,1,One
1,2,Two
2,3,Three
3,4,Four


In [28]:
df.iloc[[0], [0, 1]]

,Num,word
0,1,One


In [28]:
df.iloc[[0], [0, 1]]

,Num,word
0,1,One


In [26]:
~np.array([True, False])

array([False,  True])

In [12]:
# SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\scrublet\5.12.20'
SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
ROW_SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\Data\droplet_seq\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\new_data_3.10.21_outputs\inferCNV\update_runs\4.11.21'
not_investigate = []#['M133', 'M108']
samples = [subfolder for subfolder in os.listdir(SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
cross_check_df = None
for sample_id in samples:
#     print(sample)
    rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}'),
                                    cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}'))
    
    # REMOVING CELLS THAT SHOULD BE REMOVED
    rna_sample = rna_sample.filter_cells_by_property('should_be_removed', False)
    
    df = get_informative_df(rna_sample)
    if cross_check_df is None:
        cross_check_df = df
    else:
        cross_check_df = cross_check_df.add(df)

cross_check_df

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\Technion studies\\Keren Laboratory\\Data\\droplet_seq\\ROW_DATA'

In [6]:
sample_id = 'M120'
ROW_SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\Data\droplet_seq\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\outputs\inferCNV\update_runs\4.3.21'


rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}.pkl'),
                                    cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}.pkl'))
# rna_sample = rna_sample.filter_cells_by_property('should_be_removed', False)

In [8]:
rna_sample.number_of_cells

4613

In [9]:
rna_sample.filter_cells_by_property('should_be_removed', False).number_of_cells

4127

In [14]:
# SAMPLES_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\scrublet\5.12.20'
SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\Data\droplet_seq\new_data_3.10.21\ROW_DATA'
ROW_SAMPLES_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\Data\droplet_seq\new_data_3.10.21\ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\outputs\new_data_3.10.21_outputs\inferCNV\update_runs\4.11.21'

not_investigate = []#['M133', 'M108']
samples = [subfolder for subfolder in os.listdir(SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
# cross_check_df = None
total_n_cells = 0
for sample_id in samples:
    print(f'\n{sample_id}')
    rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, f'{sample_id}'),
                                    cells_information_path=join(SAMPLES_INFORMATION_PATH, f'{sample_id}'))
    print(f'sample cells: {rna_sample.number_of_cells}')
    total_n_cells += rna_sample.number_of_cells
    print(f'accu n cells: {total_n_cells}')
    # REMOVING CELLS THAT SHOULD BE REMOVED
#     rna_sample = rna_sample.filter_cells_by_property('should_be_removed', False)
    
    


M147.pkl
sample cells: 9173
accu n cells: 9173

M148.pkl
sample cells: 14154
accu n cells: 23327

M149.pkl
sample cells: 5025
accu n cells: 28352

M150.pkl
sample cells: 4175
accu n cells: 32527

M151.pkl
sample cells: 15649
accu n cells: 48176

M152.pkl
sample cells: 2831
accu n cells: 51007

M153.pkl
sample cells: 5798
accu n cells: 56805

M154.pkl
sample cells: 6110
accu n cells: 62915

M155.pkl
sample cells: 831
accu n cells: 63746

M156.pkl
sample cells: 17527
accu n cells: 81273

M157.pkl
sample cells: 9034
accu n cells: 90307

M158.pkl
sample cells: 7268
accu n cells: 97575

M159.pkl
sample cells: 7946
accu n cells: 105521

M160.pkl
sample cells: 9983
accu n cells: 115504

M161.pkl
sample cells: 2974
accu n cells: 118478

M162.pkl
sample cells: 11529
accu n cells: 130007

M163.pkl
sample cells: 4115
accu n cells: 134122

M164.pkl
sample cells: 8305
accu n cells: 142427

M167.pkl
sample cells: 3751
accu n cells: 146178

M168.pkl
sample cells: 5270
accu n cells: 151448

M169.pkl
